In [1]:
!pip install shapely geopandas
print("Hello World")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 9.4 MB/s eta 0:00:00
Hello World


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from shapely import wkb
from geopandas import GeoDataFrame

spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "512m").getOrCreate()

#dataframe = spark.read.format("jdbc").options(url="jdbc:postgresql://db:5432/datascience?user=postgres&password=mantequilla", dbtable="rutas",driver="org.postgresql.Driver").load()
buses = spark.read.option("header",True).csv("./data/buses.csv")
print("Hola")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/14 22:36:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Hola


In [3]:
paraderos2 = spark.read.option("header",True).csv("./data/paraderos2.csv")
paraderosrutas = spark.read.option("header",True).csv("./data/paraderosrutas.csv")

In [4]:
# Observacion: Hay que cambiar la obtencion de parder_geom, puesto que debiese venir de la tabla paraderos2 pero el csv de stops.csv venía sin ese campo
temp = paraderosrutas.join(buses, buses.vendor == paraderosrutas.cod_sinser).select(paraderosrutas.simt, 
    buses.bus, buses.fechahora, buses.lat, buses.lng, buses.velocidad, buses.pasajeros, buses.vendor, buses.id, buses.geom.alias("geom_bus"),
    paraderosrutas.servicio, paraderosrutas.nombre_par, paraderosrutas.geom.alias("parder_geom"))

print(paraderos2.columns)

print(temp.columns)
data = temp.join(paraderos2, paraderos2.stop_id == temp.simt, 'inner').select(temp.columns + [paraderos2.stop_id.alias('parader'), 
    paraderos2.stop_name, paraderos2.stop_lat.alias('parder_lat'), paraderos2.stop_lon.alias('parder_lon'), temp.geom_bus
])
#
print(data.columns)
#busesparaderos = data.withColumn("d", wkb.loads(data.geom_bus, hex=True))
data.show(5)
data.printSchema()
#fila = data.first()
#print(fila)

['stop_id', 'stop_code', 'stop_name', 'stop_lat', 'stop_lon', 'stop_url', 'wheelchair_boarding', 'geom']
['simt', 'bus', 'fechahora', 'lat', 'lng', 'velocidad', 'pasajeros', 'vendor', 'id', 'geom_bus', 'servicio', 'nombre_par', 'parder_geom']
['simt', 'bus', 'fechahora', 'lat', 'lng', 'velocidad', 'pasajeros', 'vendor', 'id', 'geom_bus', 'servicio', 'nombre_par', 'parder_geom', 'parader', 'stop_name', 'parder_lat', 'parder_lon', 'geom_bus']


+-----+-------+--------------------+-----------------+-----------------+---------+---------+------+--------+--------------------+--------+--------------------+--------------------+-------+--------------------+-----------------+-----------------+--------------------+
| simt|    bus|           fechahora|              lat|              lng|velocidad|pasajeros|vendor|      id|            geom_bus|servicio|          nombre_par|         parder_geom|parader|           stop_name|       parder_lat|       parder_lon|            geom_bus|
+-----+-------+--------------------+-----------------+-----------------+---------+---------+------+--------+--------------------+--------+--------------------+--------------------+-------+--------------------+-----------------+-----------------+--------------------+
|PE329|BJFW-53|2018-06-13 00:00:...|-33.5598373413086|-70.5652313232422|        0|        0|  T352|10006092|0101000020E610000...|    E02I|Parada 3 / (M) Tr...|0101000020E610000...|  PE329|PE329-Parad

In [5]:
from pyspark.sql.types import StringType
from pyspark.pandas import read_csv
udf = F.udf
@udf(returnType=StringType())
def add_one(x):
    if x is not None:
        return x + " another string"

#busesparaderos = data.withColumn("d", wkb.loads(data.geom_bus, hex=True))
#geo_df = GeoDataFrame.from_file("./data/paraderos2.csv")
#paraderos2 = spark.read.option("header",True).csv("./data/paraderos2.csv")
#paraderosrutas = spark.read.option("header",True).csv("./data/paraderosrutas.csv")
print(type(data))
#datos = data.groupBy().agg(add_one(F.col("velocidad"))).collect()


#data.show()
datos = data.to_pandas_on_spark()
#datos = read_csv("./data/paraderos2.csv")
print(type(datos), type(data))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.pandas.frame.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'>


In [16]:
import pyspark.pandas as ps
print(datos.columns)
tipos = data.dtypes

print(tipos)
insert_tipos = [(item, int) for item,_ in tipos]
print(insert_tipos)

#datos.groupBy().apply()
#datos.loc["geom_bus"]

#columnas = datos.get('bus')
# ('simt', str), ('bus', str), ('fechahora', str), ('lat', str), ('lng', str), ('velocidad', str), ('pasajeros', str), ('vendor', str), ('id', str), ('geom_bus', str), ('servicio', str), ('nombre_par', str), ('parder_geom', str), ('parader', str), ('stop_name', str), ('parder_lat', str), ('parder_lon', str), ('geom_bus', str)
def count(item: ps.DataFrame) -> ps.DataFrame:
    # this assumes the data is already partitioned
    id = df.iloc[0]["id"]
    count = df.shape[0]
    return pd.DataFrame({"id": [id], "count": [count]})

lotes = datos.rolling(100000)
print(type(lotes))

busesparaderos = data.withColumn("d", data.id + " Hola")
busesparaderos.printSchema()
busesparaderos.head()
#busesparaderos = datos.apply(count , axis=1)



Index(['simt', 'bus', 'fechahora', 'lat', 'lng', 'velocidad', 'pasajeros',
       'vendor', 'id', 'geom_bus', 'servicio', 'nombre_par', 'parder_geom',
       'parader', 'stop_name', 'parder_lat', 'parder_lon', 'geom_bus'],
      dtype='object')
[('simt', 'string'), ('bus', 'string'), ('fechahora', 'string'), ('lat', 'string'), ('lng', 'string'), ('velocidad', 'string'), ('pasajeros', 'string'), ('vendor', 'string'), ('id', 'string'), ('geom_bus', 'string'), ('servicio', 'string'), ('nombre_par', 'string'), ('parder_geom', 'string'), ('parader', 'string'), ('stop_name', 'string'), ('parder_lat', 'string'), ('parder_lon', 'string'), ('geom_bus', 'string')]
[('simt', <class 'int'>), ('bus', <class 'int'>), ('fechahora', <class 'int'>), ('lat', <class 'int'>), ('lng', <class 'int'>), ('velocidad', <class 'int'>), ('pasajeros', <class 'int'>), ('vendor', <class 'int'>), ('id', <class 'int'>), ('geom_bus', <class 'int'>), ('servicio', <class 'int'>), ('nombre_par', <class 'int'>), ('parder_g

Row(simt='PE329', bus='BJFW-53', fechahora='2018-06-13 00:00:10+00', lat='-33.5598373413086', lng='-70.5652313232422', velocidad='0', pasajeros='0', vendor='T352', id='10006092', geom_bus='0101000020E6100000010000C02CA451C0010000C0A8C740C0', servicio='E02I', nombre_par='Parada 3 / (M) Trinidad', parder_geom='0101000020E61000001FB50FE9BB841541E84D13C890FB5741', parader='PE329', stop_name='PE329-Parada 3 / (M) Trinidad', parder_lat='-33.5472232837671', parder_lon='-70.5881270000095', geom_bus='0101000020E6100000010000C02CA451C0010000C0A8C740C0', d=None)

In [ ]:
#geo_df.head()

In [ ]:
#print(buses.schema.jsonValue())
#minimos = busesparaderos.groupBy('fechahora').agg(F.min(busesparaderos.d).alias("md")).select(busesparaderos.distance, busesparaderos.fechahora.alias("fh"))
#resultados = busesparaderos.join(minimos, busesparaderos.d == minimos.md & busesparaderos.fechahora == minimos.fh, 'inner')

In [ ]:
def get_Bus_Cant_Prom_Por_Paredero():#Frecuenica
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']

    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']
    #*************************
    #print(type(result.index))
    #print(result.index.names)
    #print(result.index.values)
    #print(result.index.levels)

    dataframe_ruta = pd.read_csv("./E04I.csv")
    temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    list_paredors = temp2.index.levels[0]
    #temp3 = temp2.loc['PA146']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]

    dataframe_buses = {'paradero':[],'promedio_bus_hora':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    #print(len(Hour_list_from))
    #print(len(Hour_list_to))

    for i in list_paredors: 
        x = temp2.loc[i]
        List_total_buses = list()
        geo = x['parder_geom'].iat[0]
        geo_lat = x['parder_lat'].iat[0]
        geo_lon = x['parder_lon'].iat[0]
        paradero_list = list()
        #print(geo)
        for j in range(23):
            #print(j)
            df = x[(x['fechahora'] >= Hour_list_from[j]) & (x['fechahora'] <= Hour_list_to[j])]
            List_total_buses.append(df['bus'].nunique())
            #Lista_interval.append(Hour_list_from[j]+' to '+Hour_list_to[j])
        mean = (sum(List_total_buses) / len(List_total_buses))
        dataframe_buses['paradero'].append(i)
        dataframe_buses['promedio_bus_hora'].append(mean)
        dataframe_buses['geo_parader'].append(geo)
        dataframe_buses['geo_parader_lat'].append(geo_lat)
        dataframe_buses['geo_parader_lon'].append(geo_lon)
        if mean < 5:
            dataframe_buses['calificacion'].append(0)
        elif mean > 6:
            dataframe_buses['calificacion'].append(1)
        else:
            dataframe_buses['calificacion'].append(2)


        
        #dataframe_buses['rango_hora'] += Lista_interval


    #dataframe_total_busparadero = pd.DataFrame(dataframe_buses)
    #print(dataframe_total_busparadero)
    #dataframe_total_busparadero.to_csv('E04I_NOTED.csv')